DGM

In [26]:
import os
import yaml
import torch
import pickle
import numpy as np
from models.mart import MART
from types import SimpleNamespace
from torch.utils.data import Dataset

In [33]:
!nvidia-smi


Sat Jan 11 23:23:07 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [34]:
# Install the correct version of PyTorch with GPU support
!pip install torch==2.1.0+cu118 torchvision==0.15.0+cu118 torchaudio==2.1.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 673.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 83.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==2.1.0+cu118 and torchvision==0.15.0+cu118 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==2.1.0+cu118
    torchvision 0.15.0+cu118 depends on torch==2.0.0+cu118

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#deali

In [35]:
import torch
print(torch.cuda.is_available())

True


In [58]:
def load_config(config_path):
    """
    Load configuration from a YAML file.

    Args:
        config_path (str): Path to the YAML configuration file.

    Returns:
        dict: Loaded configuration dictionary.
    """
    print(f"[INFO] Loading configuration from {config_path}...")
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    print("[INFO] Configuration loaded successfully!")
    return config


In [59]:
CHECKPOINT_PATH = "files/sdd_ckpt_best.pth"  # Update this
TEST_DATA_PATH = "files/sdd_test.pkl"  # Update this
CONFIG_PATH = "files/mart_sdd_reproduce.yaml"  # Update this
BATCH_SIZE = 64

In [65]:
def load_model(checkpoint_path, opts):
    print("[INFO] Loading model from checkpoint...")
    args = SimpleNamespace(**opts)
    model = MART(args)  # Remove .cuda()
    checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=True)


    model.load_state_dict(checkpoint['state_dict'])  # or whatever loading method you are using

    print("[INFO] Model loaded successfully!")
    return model


In [61]:
def evaluate_model(model, test_loader, opts):
    print("[INFO] Starting evaluation...")
    ade_sum, fde_sum, total_agents = 0.0, 0.0, 0

    # Set the device to GPU or CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move model to device

    with torch.no_grad():
        for sample in test_loader:
            x_abs, y = sample
            x_abs, y = x_abs.to(device), y.to(device)  # Move tensors to device

            batch_size, num_agents, length, _ = x_abs.size()

            x_rel = torch.zeros_like(x_abs).to(device)  # Ensure x_rel is on the same device
            x_rel[:, :, 1:] = x_abs[:, :, 1:] - x_abs[:, :, :-1]
            x_rel[:, :, 0] = x_rel[:, :, 1]

            y_pred = model(x_abs, x_rel)

            if opts["pred_rel"]:
                cur_pos = x_abs[:, :, [-1]].unsqueeze(2)
                y_pred = torch.cumsum(y_pred, dim=3) + cur_pos

            ade = torch.min(torch.mean(torch.norm(y_pred - y[:, :, None], dim=-1), dim=3), dim=2)[0].mean().item()
            fde = torch.min(torch.mean(torch.norm(y_pred[:, :, :, -1:] - y[:, :, None, -1:], dim=-1), dim=3), dim=2)[0].mean().item()

            ade_sum += ade * num_agents * batch_size
            fde_sum += fde * num_agents * batch_size
            total_agents += num_agents * batch_size

    ade_avg = (ade_sum / total_agents) * opts["scale"]
    fde_avg = (fde_sum / total_agents) * opts["scale"]

    print(f"[INFO] Evaluation Results: ADE = {ade_avg:.4f}, FDE = {fde_avg:.4f}")
    return ade_avg, fde_avg


In [62]:
import torch
import numpy as np

class TrajectoryDataset(Dataset):
    def __init__(
        self, obs_len=8, pred_len=12, mode='train', scale=10, inputs=None, max_agents=50
    ):
        super(TrajectoryDataset, self).__init__()

        self.obs_len = obs_len
        self.pred_len = pred_len
        self.seq_len = self.obs_len + self.pred_len
        self.scale = scale
        self.max_agents = max_agents

        with open('files/sdd_test.pkl'.format(mode), 'rb') as f:
            traj = pickle.load(f)

        traj_tmp = []

        for t in traj:
            traj_tmp.append(t)
            if mode == 'train':
                traj_tmp.append(np.flip(t, axis=1))

        self.traj = []
        if 'pos_x' in inputs and 'pos_y' in inputs:
            for t in traj_tmp:
                t -= t[:, :1, :]
                self.traj.append(t)
        else:
            self.traj = traj_tmp


    def __len__(self):
        return len(self.traj)

    def __getitem__(self, index):
        past_traj = self.traj[index][:, :self.obs_len] * self.scale
        future_traj = self.traj[index][:, self.obs_len:] * self.scale

        # Padding the number of agents to max_agents if necessary
        num_agents = past_traj.shape[0]
        if num_agents < self.max_agents:
            pad_size = self.max_agents - num_agents
            past_traj_padded = np.pad(past_traj, ((0, pad_size), (0, 0), (0, 0)), mode='constant')
            future_traj_padded = np.pad(future_traj, ((0, pad_size), (0, 0), (0, 0)), mode='constant')
        else:
            past_traj_padded = past_traj
            future_traj_padded = future_traj

        past_traj = torch.from_numpy(past_traj_padded).type(torch.float)
        future_traj = torch.from_numpy(future_traj_padded).type(torch.float)

        return [past_traj, future_traj]


In [67]:
from torch.utils.data import DataLoader
if __name__ == "__main__":
    opts = load_config(CONFIG_PATH)
    model = load_model(CHECKPOINT_PATH, opts)

    # Prepare the test dataset and DataLoader
    test_dataset = TrajectoryDataset(
        obs_len=opts["past_length"],
        pred_len=opts["future_length"],
        mode="test",  # This loads the test set
        scale=opts["scale"],
        inputs=opts["inputs"]
    )
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    print("[INFO] Starting evaluation with test data...")
    evaluate_model(model, test_loader, opts)

[INFO] Loading configuration from files/mart_sdd_reproduce.yaml...
[INFO] Configuration loaded successfully!
[INFO] Loading model from checkpoint...
[INFO] PRT Agg: cat
[INFO] HRT Agg: avg
[INFO] Binary Threshold Function Type: 2
[INFO] Model loaded successfully!
[INFO] Starting evaluation with test data...
[INFO] Starting evaluation...
[INFO] Evaluation Results: ADE = 0.7596, FDE = 1.2068
